# Water Quality Monitoring Software

The program displays the water quality in the area when enetered zipcode.
* Input: zipcode
* output: Name of the water supplier
          area served
          zipcode
          contaminants count
          violation count
          hardnesslevel
          precautions
          past 5 year trends in that area.
          
Author: Aparna Ayyalasomayajula
          

In [ ]:


# import tkinter, mysql connector
try:
    from tkinter import *
    import tkinter as tk                # python 3
    from tkinter import font as tkfont  # python 3
    from tkinter import ttk
    from tkinter.messagebox import *
    import mysql.connector
    from mysql.connector import errorcode
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)
    # Implement the default Matplotlib key bindings.
    from matplotlib.backend_bases import key_press_handler
    from matplotlib.figure import Figure

    
#python import error if any module is not found
except ImportError:
    import Tkinter as tk     # python 2
    import tkFont as tkfont  # python 2
    
#title of project
title='Water Quality Monitoring Software'
#main class
class WaterQualityManagementApp(tk.Tk):
    #function to initialise tkinter and other frames in application
    def __init__(self, *args, **kwargs):
        tk.Tk.__init__(self, *args, **kwargs)
        
#set title font and size
        self.title_font = tkfont.Font(family='Helvetica', size=18, weight="bold", slant="italic")
        # in a container we are stacking all frames one above the other 
        # put all of the pages in the same location;
        # the one on the top of the stacking order will be the one that is visible.
        #assigning TK frame to current class
        container = tk.Frame(self)
        #in frame, initialise grid row and gid column
        container.pack(side="top", fill="both", expand=True)
        container.grid_rowconfigure(0, weight=1)
        container.grid_columnconfigure(0, weight=1)

        self.frames = {}
        #looping each frames initialising and assign respective widgets to the frame.
        for F in (StartPage, UserSearchPage, AdminSearch,AdminLogin,RegisterNewUser):
            page_name = F.__name__
            print(page_name)
            frame = F(parent=container, controller=self,arg=None)
            self.frames[page_name] = frame

            
            frame.grid(row=0, column=0, sticky="nsew")
        #default frame "start page"
        self.show_frame("StartPage",None)
        #create and call the databse instance.
        obj_tableMetadata=tableMetadata()
        #calling the db creation function
        obj_tableMetadata.db_creation()
        #calling the table creation funtion; creating initial table schema in db
        obj_tableMetadata.table_Creation()
    
    #this function is used to call whichever frame we want to display
    def show_frame(self, page_name,arg):
        frame = self.frames[page_name]
        
        #if condition to check if usersearchpage or adminserachpage
        if(page_name=='UserSearchPage' or page_name=='AdminSearch' ):
            global title    # Needed to remove
            #calling setname function to destroy duplicate widgets
            frame.setname(arg)
            
        frame.tkraise()

#application start page class
class StartPage(tk.Frame):
    # defined initial function, adding zipcode search widgets and search buttons
    def __init__(self, parent, controller,arg):
        tk.Frame.__init__(self, parent)
        self.controller = controller
        lblstartearchheader = tk.Label(self,name='lblstartearchheader', text="Enter the Zipcode to check water quality in your area",font = ('Helvetica', 11, 'bold'))
        lblstartearchheader.grid(column=1, row=2, sticky=(N, S, E, W))
        label1 = tk.Label(self, text="Please Enter ZipCode")
        label1.grid(column=0, row=3, sticky=(N, S, E, W))

        inputtxt = tk.Text(self,height = 2,width = 27)
        inputtxt.grid(column=1, row=3, sticky=(N, S, E, W))
                
                
        #inputtxt.pack()

        button1 = tk.Button(self, text="Search",
                            command=lambda: controller.show_frame("UserSearchPage",[inputtxt.get("1.0","end-1c")]))
        #button1.pack()
        button1.grid(column=2, row=3, sticky=(N, S, E, W))
        button2 = tk.Button(self, text="Login As Admin",
                            command=lambda: controller.show_frame("AdminLogin",[inputtxt]))
       
        #button2.pack()
        button2.grid(column=6, row=2, sticky=(N, S, E, W),padx=100)
        

class AdminLogin(tk.Frame):
    global username_verify
    global password_verify
    def __init__(self, parent, controller,arg):
        #print(arg)
        tk.Frame.__init__(self, parent)
        #print(arg)
        self.controller = controller
        self.title='Admin Login'
        self.controller.title("Admin Login")
        
        
        Label(self, text="Please enter details below to login").pack()
        Label(self, text="").pack()

        username_verify = StringVar()
        password_verify = StringVar()
        Label(self, text="Username * ").pack()
        username_login_entry = Entry(self, textvariable=username_verify)
        username_login_entry.pack()
        Label(self, text="").pack()
        Label(self, text="Password * ").pack()
        password__login_entry = Entry(self, textvariable=password_verify, show= '*')
        password__login_entry.pack()
        Label(self, text="").pack()
        Button(self, text="Login", width=10, height=1, command=lambda:self.login_verification(username_verify.get(),password_verify.get())).pack()
        
    def login_verification(self,username,password):
        print(username)
        print(password)
        count=0
        try:
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
            #cnx = mysql.connector.connect(user='root',password='Unlv@123',host='127.0.0.1',database='waterquality')
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            query = ("select id,UserName,Password from waterquality.useradmindetails WHERE UserName = %s and Password = %s ")
            num_items=cursor.execute(query ,(username, password))
            
            #cursor.execute(query)
            print('--rc')
            count=cursor.fetchall()
            
            print(cursor.rowcount)
            if(cursor.rowcount==0):
                tk.messagebox.showerror("showerror", "Not a valid login details")
                cursor.close()
                cnx.close()
            else:
                
                self.controller.show_frame("AdminSearch",[username])
                cursor.close()
                cnx.close()

class UserSearchPage(tk.Frame):
    ival=1
    nam=''
    def __init__(self, parent, controller,arg):
        tk.Frame.__init__(self, parent)
#         print('k')
#         print(self)
        self.controller = controller
        ival=0
        self.title='User Search'
    #destroying widgests in frame to avoid duplicates. and creating new widgets everytime    
    def setname(self,arg):
       
        global title    # Needed to modify global copy of globvar
        
        print(self.winfo_children())  
        print(len(self.winfo_children())) 
        if(len(self.winfo_children())>2):
            for child in self.winfo_children():
                print(child)
                child.destroy()

            
        title = 'User Search' 
        button_homepage = tk.Button(self, text="Go to the Home page",name='btnhomepage',
                           command=lambda: self.controller.show_frame("StartPage",['b']))
        button_homepage.grid(row=0,column=2, sticky=( W),padx=100)
        label_enterzipcode = tk.Label(self, text="Entered Zipcode",name='lblenter')
        label_enterzipcode.grid(row=2,column=0,sticky=(W),padx=100)
       
        input_userzipcode = tk.Text(self, height = 2,
                width = 25,
                bg = "light yellow",name='userzipcode')
        
        input_userzipcode.insert('end', arg[0])
        input_userzipcode.grid(row=2,column=1,ipady=10)
        #buttonzipsearch = tk.Button(self, text="Search",name='btnuserzipsearch', height = 2,
                           #command=lambda: self.controller.show_frame("StartPage",['b']))
        #buttonzipsearch.grid(row=2,column=2,ipady=10)
        
        

        # Set the size of the tkinter window
        

        # Create an instance of Style widget
        

        # Add a Treeview widget
        db=DatabaseConnectivity()
        
        getpd= db.fetchArecodedata(arg[0])
        print(getpd)
        print(getpd.columns.values.tolist())
        tree_Areacode = ttk.Treeview(self)
        tree_Areacode.grid(row=3,column=0,columnspan=2, sticky="nsew",)
        tree_Areacode['columns'] = getpd.columns.values.tolist()
        tree_Areacode['show'] = 'headings'

        for i in getpd.columns.values.tolist():
            tree_Areacode.column(i, width = 100,stretch=True)
            tree_Areacode.heading(i, text = i)
        
        
        for indexs, row in getpd.iterrows():
            tree_Areacode.insert("", 'end', text = indexs, values = list(row))
        #print(getpd.iterrows())


        def edit():
           # Get selected item to Edit
            print(tree_Areacode.selection())
            selected_item = tree_Areacode.selection()[0]
            print(tree_Areacode.item(selected_item)['values'][1])
            print(tree_Areacode.item(selected_item)['values'][0])
            #tree1.item(selected_item, text="blub", values=("foo", "bar"))

        def delete():
           # Get selected item to Delete
            selected_item = tree_Areacode.selection()[0]
            tree_Areacode.delete(selected_item)
        def showtrends():
            if(len(self.winfo_children())>2):
                for child in self.winfo_children():
                    if(str(child)=='.!frame.!usersearchpage.trendtreeview'):
                        child.destroy()
                    

                #child.destroy()
            if(len(tree_Areacode.selection())>0):
                selected_item = tree_Areacode.selection()[0]
                zipval=tree_Areacode.item(selected_item)['values'][1]
                db=DatabaseConnectivity()
                gettrends= db.fetchtrends(zipval)
                print(gettrends.columns.values.tolist())
                trendtreeview = ttk.Treeview(self,name='trendtreeview')
                trendtreeview.grid(row=6,column=0,columnspan=2, sticky="nsew")
                trendtreeview['columns'] = gettrends.columns.values.tolist()
                trendtreeview['show'] = 'headings'
                for i in gettrends.columns.values.tolist():
                    trendtreeview.column(i, width = 100,stretch=True)
                    trendtreeview.heading(i, text = i)
                for indexs, row in gettrends.iterrows():
                    #trendtreeview.insert("", 'end', text = indexs, values = list(row))
                    trendtreeview.insert("", 'end', text = indexs, values = list(row))
                print('-----')
                print(gettrends)
                print('-----')
                print(gettrends.size)
                print('-====-')

                #Dataf=pd.read_csv('convertcsv (1).csv')

                #dft = seaborn.load_dataset(gettrends)
  
                ############# Main Section         #############
                # Form a facetgrid using columns with a hue
                 #select Ac.AreacodeId,Ac.Zipcodes,Ac.Area,vd.ViolationCount,vd.ContaminationCount ,vd.Year,vd.HardnessLevel from waterquality.AreacodeDetails as AC inner join waterquality.ViolationHistoricalData as  vd  on vd.AreaCodeId=AC.AreacodeId where zipcodes 
                #graph = sns.FacetGrid(gettrends, row ='ViolationCount')
                # map the above form facetgrid with some attributes
                #graph.map(sns.histplot,'Year')
                #graph = sns.barplot(x="Year", y=["ViolationCount",'ContaminationCount'], data=gettrends)

                # show the object
                #plt.show()
                
                if(gettrends.size>0):
                    print(gettrends['ViolationCount'])
                    df = pd.DataFrame({'ViolationCount': gettrends['ViolationCount'].to_numpy(),
                                       'ContaminationCount': gettrends['ContaminationCount'].to_numpy(),'HardnessLevel': gettrends['HardnessLevel'].to_numpy()}, index=gettrends['Year'])
                    ax = df.plot.bar(rot=0)


                    #canvas = FigureCanvasTkAgg(graph.fig,self)  # A tk.DrawingArea.
                    canvas = FigureCanvasTkAgg(ax.figure,self) 
                    canvas.draw()
                    canvas.get_tk_widget().grid(row = 3, column = 2)
                    
                    print('---min')
                    print(gettrends['ViolationCount'].to_numpy().min())
                    minmaxCounts='Min violation count :  '+str(gettrends['ViolationCount'].to_numpy().min()) + '\n Max violation count : '+str(gettrends['ViolationCount'].to_numpy().max())+ '\n Min Contamination Count :  '+str(gettrends['ContaminationCount'].to_numpy().min()) + '\n Max Contamination Count : '+str(gettrends['ContaminationCount'].to_numpy().max()) +'\n Min Hardness Level :  '+str(gettrends['HardnessLevel'].to_numpy().min()) + '\n Max Hardness Level : '+str(gettrends['HardnessLevel'].to_numpy().max()) 
                    label_minmaxviolationcount = tk.Label(self, text=minmaxCounts,name='lblminmaxviolation')
                    label_minmaxviolationcount.grid(row=5,column=2)
#            
                    
                    
                    


            else:
                tk.messagebox.showerror("Error", "Please select a record")    
            
        def showprecaution():
            if(len(tree1.selection())>0):
                selected_item = tree1.selection()[0]
                precaution=tree1.item(selected_item)['values'][6]
                lblprecaution = tk.Label(self,name='lblprecaution', text="Selected Precaution:" +precaution )
                lblprecaution.grid(row=7,column=0, sticky="nsew",padx=100)
                #print('a')
            else:
                tk.messagebox.showerror("Error", "Please select a record") 

        # Add Buttons to Edit and Delete the Treeview items
        showtrend_btn = ttk.Button(self, text="Show Trendz", command=showtrends,width=20)
        showtrend_btn.grid(row=5,column=0,columnspan=2 , sticky=( E))
        showPrecaution_btn = ttk.Button(self, text="Show Precaution", command=showprecaution,width=20)
        showPrecaution_btn.grid(row=5,column=1,columnspan=2, sticky=( W))

        #setattr(self, 'nam', arg[0])

class AdminSearch(tk.Frame):

    nam=''

    def __init__(self, parent, controller,arg):
        tk.Frame.__init__(self, parent)
        #print('k')
        #print(self)
        self.controller = controller
        lbladminsearchheader = tk.Label(self,name='lbladminsearchheader', text="Admin Search")
        lbladminsearchheader.grid(row=0,column=0, sticky="nsew",padx=100)
       
    def setname(self,arg):
        print('arg')
        print(tk)
        global title    # Needed to modify global copy of globvar
        title = 'User Search' 
        zip_verify = StringVar()
        if(len(self.winfo_children())>2):
            for child in self.winfo_children():
               
                if(str(child)=='.!frame.!adminsearch.lbladminsearchheader' 
                   or str(child)=='.!frame.!adminsearch.lbladminsearchheaderuser'
                   or str(child)=='.!frame.!adminsearch.btnstartpage' 
                   or child=='.!frame.!adminsearch.btnregisternewuser' 
                   or str(child)=='.!frame.!adminsearch.buttonzip' 
                   or str(child)=='.!frame.!adminsearch.lbladminenterzip' 
                   or str(child)=='.!frame.!adminsearch.buttonfile' 
                   or  str(child)=='.!frame.!adminsearch.txtfile' 
                   or  str(child)=='.!frame.!adminsearch.lbladminenterfile'):
                    print('a')
                else:
                    child.destroy()
       
    def setname(self,arg):
        logusername="Logged in user:"
        if(arg != None and len(arg)>0  and len(arg[0])>0):
            logusername=logusername+arg[0]
        else:
            print('a')
            
            
        lbladminsearchheaderuser = tk.Label(self,name='lbladminsearchheaderuser',font = ('Helvetica', 18, 'bold'), text=logusername)
        lbladminsearchheaderuser.grid(row=1,column=4, sticky="nsew")
        btnstartpage = tk.Button(self, text="Go to the start page",name='btnstartpage',
                           command=lambda: self.controller.show_frame("StartPage",['b']))
        btnstartpage.grid(row=2,column=3, sticky="nsew")
        zip_verify = StringVar()
        file_path=StringVar()
        btnregisternewuser= tk.Button(self,name='btnregisternewuser', text="Register New Admin", width=10, height=1, command=lambda:self.controller.show_frame("RegisterNewUser",[logusername]))
        btnregisternewuser.grid(row=2,column=4, sticky="nsew")
        lbladminenterzip = tk.Label(self,name='lbladminenterzip', text="Enter Zip code")
        lbladminenterzip.grid(row=4,column=0, sticky="nsew")
        zip_login_entry = Entry(self,textvariable=zip_verify,name='txtzip')
        zip_login_entry.grid(row=4,column=1, sticky="nsew")
        buttonzip = tk.Button(self,name='buttonzip' ,text="Search", 
                            command=lambda: self.showtree(zip_verify.get()))
        buttonzip.grid(row=4,column=2, sticky="nsew")
        
        lbladminenterfile = tk.Label(self,name='lbladminenterfile', text="File Path")
        lbladminenterfile.grid(row=5,column=0, sticky="nsew")
        file_login_entry = Entry(self,textvariable=file_path,name='txtfile')
        file_login_entry.grid(row=5,column=1, sticky="nsew")
        buttonfile = tk.Button(self,name='buttonfile' ,text="Upload Trends", 
                            command=lambda:uploadtrendsfile(file_path.get()))
        buttonfile.grid(row=5,column=2, sticky="nsew")
        Add_btn = ttk.Button(self, text="Add",name='btn_addareadetails', command=lambda:self.addata(self),width=20)
        Add_btn.grid(row=5,column=3, sticky="nsew")
        def uploadtrendsfile(FileName):
            print(FileName)
            db=DatabaseConnectivity()
            file= db.historcTrendsUpload(FileName)
            tk.messagebox.showinfo("Trends History", "Trends Data Uploaded Sucessfully")

        
        # Set the size of the tkinter window
    @staticmethod        
    def addata(self):
        areaname = StringVar()
        areaserved = StringVar()
        contcount = StringVar()
        voilationcount = StringVar()
        zipval=StringVar()
        precautionentry=StringVar()
        AreacodeIDval=StringVar()
        suppliername = StringVar()
        hardnessentryval= StringVar()
        for child in self.winfo_children():
            if(str(child)=='.!frame.!adminsearch.lbladminsearchheader' 
               or str(child)=='.!frame.!adminsearch.admintreeview'
               or str(child)=='.!frame.!adminsearch.lbladminsearchheaderuser'
               or str(child)=='.!frame.!adminsearch.btnstartpage'  or str(child)=='.!frame.!adminsearch.btn_addareadetails'
               or str(child)=='.!frame.!adminsearch.btnregisternewuser' or str(child)=='.!frame.!adminsearch.buttonzip' or str(child)=='.!frame.!adminsearch.lbladminenterzip' or  str(child)=='.!frame.!adminsearch.txtzip' or str(child)=='.!frame.!adminsearch.buttonfile' or  str(child)=='.!frame.!adminsearch.txtfile'or  str(child)=='.!frame.!adminsearch.lbladminenterfile'):
                    print()
            else:
                child.destroy()
        lblAreaHeaderName = tk.Label(self, text="Add Area Code Details:",name='lblAreaHeaderName',font = ('Helvetica', 11, 'bold'))
        lblAreaHeaderName.grid(row=9,column=0, sticky="nsew")
        
        lblName = tk.Label(self, text="Name",name='add_areaname')
        lblName.grid(row=11,column=0, sticky="nsew")
            #lblName.pack(side="left", fill="x",padx=5,pady=5)
            
        name_entry = Entry(self,textvariable=suppliername ,name='add_areanametext')
        name_entry.grid(row=11,column=1, sticky="nsew")
            
        lblAreaserved = tk.Label(self, text="Area Served",name='add_lblAreaserved')
        lblAreaserved.grid(row=12,column=0, sticky="nsew")
        areaserved_entry = Entry(self,textvariable=areaserved ,name='add_areaserved_entry')
        areaserved_entry.grid(row=12,column=1, sticky="nsew")
            
        lblcontcount = tk.Label(self, text="Contamination Count",name='add_lblcontcount')
            
        lblcontcount.grid(row=13,column=0, sticky="nsew")
        contcount_entry = Entry(self,textvariable=contcount ,name='add_contcount_entry')
        contcount_entry.grid(row=13,column=1, sticky="nsew")
        lblviolcount = tk.Label(self, text="Voilation Count",name='add_lblviolcount')
        lblviolcount.grid(row=14,column=0, sticky="nsew")
            
        violcount_entry = Entry(self,textvariable=voilationcount ,name='add_violcount_entry')
        violcount_entry.grid(row=14,column=1, sticky="nsew")
        lblzip = tk.Label(self, text="Zip",name='add_lblzip')
        lblzip.grid(row=15,column=0, sticky="nsew")
        zip_entry = Entry(self,textvariable=zipval ,name='add_zip_entry')
        zip_entry.grid(row=15,column=1, sticky="nsew")
        lblprecaution = tk.Label(self, text="Precaution",name='add_lblprecaution')
        lblprecaution.grid(row=16,column=0, sticky="nsew")
        precaution_entry = Entry(self,textvariable=precautionentry ,name='add_precaution_entry')
        precaution_entry.grid(row=16,column=1, sticky="nsew")
            
        lblHardness = tk.Label(self, text="Hardness Level",name='add_lblhardness')
        lblHardness.grid(row=17,column=0, sticky="nsew")
        hardness_entry = Entry(self,textvariable=hardnessentryval ,name='add_hardness_entry')
        hardness_entry.grid(row=17,column=1, sticky="nsew")
            
        Adddetails_btn = ttk.Button(self,name='btnAdd_entries', text="Add", command=lambda:addatatodb(suppliername.get(),areaserved.get(),contcount.get(),voilationcount.get(),zipval.get(),precautionentry.get(),hardnessentryval.get()))
        Adddetails_btn.grid(row=18,column=1, sticky="nsew")
        
        def addatatodb(AreaName,Areaserved,contcount,voilationcount,zipval,precautionentry,hardnessentryval):
            db=DatabaseConnectivity()
            addareacode= db.AddAreacodeDetails(AreaName,Areaserved,contcount,voilationcount,zipval,precautionentry,hardnessentryval)
            print(addareacode)
            tk.messagebox.showinfo("Record Added", "Record Added Sucessfully")    
        
    def showtree(self,arg):
        print('-arg')
        print(arg)
        
        if(len(self.winfo_children())>2):
            for child in self.winfo_children():
                print(child)

                if(str(child)=='.!frame.!adminsearch.lbladminsearchheader'or  str(child)=='.!frame.!adminsearch.lbladminenterzip' 
                   or  str(child) == '.!frame.!adminsearch.txtzip'  
                   or  str(child)=='.!frame.!adminsearch.txtfile' 
                   or  str(child)=='.!frame.!adminsearch.lbladminenterfile' 
                   or str(child)=='.!frame.!adminsearch.lbladminsearchheaderuser'
                   or str(child)=='.!frame.!adminsearch.btnstartpage' 
                   or str(child)=='.!frame.!adminsearch.btnregisternewuser' 
                   or str(child)=='.!frame.!adminsearch.buttonzip' 
                   or str(child)=='.!frame.!adminsearch.btn_addareadetails'
                   or str(child)=='.!frame.!adminsearch.lbladminenterfile' or
                   str(child) == '.!frame.!adminsearch.txtfile' 
                   or  str(child) == '.!frame.!adminsearch.buttonfile'):
                    print('a') 
                else:
                    child.destroy()
                    
        db=DatabaseConnectivity()
        
        getpd= db.fetchArecodedata(arg)
        print(getpd)
        print(getpd.columns.values.tolist())
#         x_scrollbar = ttk.Scrollbar(self, orient=tk.HORIZONTAL)
#         x_scrollbar.grid(row=5, column=1, sticky=tk.E+tk.W)
#         y_scrollbar = ttk.Scrollbar(self, orient=tk.VERTICAL)
#         y_scrollbar.grid(row=5, column=1, sticky=tk.N+tk.S)
        tree1 = ttk.Treeview(self,name='admintreeview')
        tree1.grid(row=6,column=0, sticky="nsew")
        tree1['columns'] = getpd.columns.values.tolist()
        tree1['show'] = 'headings'

        for i in getpd.columns.values.tolist():
            tree1.column(i, width = 100,stretch=True)
            tree1.heading(i, text = i)
        
        
        for indexs, row in getpd.iterrows():
            print(indexs)
            print(row)
            tree1.insert("", 'end', text = indexs, values = list(row))
        


        def edit():
            for child in self.winfo_children():
                if(str(child)=='.!frame.!adminsearch.lbladminsearchheader' or str(child)=='.!frame.!adminsearch.admintreeview' or str(child)=='.!frame.!adminsearch.lbladminsearchheaderuser'or str(child)=='.!frame.!adminsearch.btnstartpage' or child=='.!frame.!adminsearch.btnregisternewuser' or str(child) == '.!frame.!adminsearch.buttonzip' or str(child)=='.!frame.!adminsearch.lbladminenterzip' or  str(child)=='.!frame.!adminsearch.txtzip' or str(child)=='.!frame.!adminsearch.buttonfile'or  str(child)=='.!frame.!adminsearch.txtfile'or  str(child)=='.!frame.!adminsearch.lbladminenterfile'):
                    print()
                else:
                    child.destroy()
           # Get selected item to Edit
            if(len(tree1.selection())>0):
                selected_item = tree1.selection()[0]
                print(tree1.item(selected_item)['values'][1])
                print(tree1.item(selected_item)['values'][0])
                #tree1.item(selected_item, text="blub", values=("foo", "bar"))
                areaname = StringVar()
                suppliername = StringVar()
                areaserved = StringVar()
                contcount = StringVar()
                voilationcount = StringVar()
                zipval = StringVar()
                precautionentry = StringVar()
                hardnessentry=StringVar()

                suppliername=StringVar(self,tree1.item(selected_item)['values'][2])

                contcount=StringVar(self,tree1.item(selected_item)['values'][5])
                voilationcount=StringVar(self,tree1.item(selected_item)['values'][4])
                AreacodeIDval=StringVar(self,tree1.item(selected_item)['values'][0])
                precautionentry= StringVar(self,tree1.item(selected_item)['values'][6])
                hardnessentry=StringVar(self,tree1.item(selected_item)['values'][7])
                areaserved =StringVar(self,tree1.item(selected_item)['values'][3])
                lblName = tk.Label(self, text="Name",name='edit_areaname')
                #lblName.pack(side="left", fill="x",padx=5,pady=5)
                lblName.grid(row=8,column=0, sticky="nsew")
                name_entry = Entry(self,textvariable=suppliername ,name='edit_areanametext')
                #name_entry.insert(-1, areaname)
                #name_entry.pack(side="left",padx=5,pady=5)
                name_entry.grid(row=8,column=1, sticky="nsew")
                lblAreaserved = tk.Label(self, text="Area Served",name='edit_lblAreaserved')
                #lblAreaserved.pack(side="left", fill="x",padx=5,pady=10)
                lblAreaserved.grid(row=9,column=0, sticky="nsew")
                areaserved_entry = Entry(self,textvariable=areaserved ,name='edit_areaserved_entry')
                #areaserved_entry.pack(side="left",padx=5,pady=5)
                areaserved_entry.grid(row=9,column=1, sticky="nsew")
                lblcontcount = tk.Label(self, text="Contamination Count",name='edit_lblcontcount')
                lblcontcount.grid(row=10,column=0, sticky="nsew")
                #lblcontcount.pack(side="left", fill="x",padx=5,pady=5)
                contcount_entry = Entry(self,textvariable=contcount ,name='edit_contcount_entry')
                #contcount_entry.insert(-1, contcount)
                #contcount_entry.pack(side="left",padx=5,pady=5)
                contcount_entry.grid(row=10,column=1, sticky="nsew")
                lblviolcount = tk.Label(self, text="Voilation Count",name='edit_lblviolcount')
                lblviolcount.grid(row=11,column=0, sticky="nsew")
                #lblviolcount.pack(side="left", fill="x",padx=5,pady=5)
                violcount_entry = Entry(self,textvariable=voilationcount ,name='edit_violcount_entry')
                #violcount_entry.insert(-1, voilationcount)
                #violcount_entry.pack(side="left",padx=5,pady=5)
                violcount_entry.grid(row=11,column=1, sticky="nsew")
                lblprecaution = tk.Label(self, text="Precaution",name='edit_lblprecaution')
                lblprecaution.grid(row=12,column=0, sticky="nsew")
                precaution_entry = Entry(self,textvariable=precautionentry ,name='edit_precaution_entry')
                precaution_entry.grid(row=12,column=1, sticky="nsew")
                lblhardness = tk.Label(self, text="Hardness Level",name='edit_lblhardness')
                lblhardness.grid(row=13,column=0, sticky="nsew")
                hardness_entry = Entry(self,textvariable=hardnessentry ,name='edit_Hardness_entry')
                hardness_entry.grid(row=13,column=1, sticky="nsew")
                updatedetails_btn = ttk.Button(self, text="Update", command=lambda:updatetatodb(areaname.get(),areaserved.get(),contcount.get(),voilationcount.get(),zipval.get(),AreacodeIDval.get(),precautionentry.get(),hardnessentry.get()))
                #updatedetails_btn.pack(side="left",padx=20,pady=30)
                updatedetails_btn.grid(row=14,column=1, sticky="nsew")
            else:
                tk.messagebox.showerror("Error", "Please Select record")
#                 Add_btn = ttk.Button(self, text="Add", command=addata,width=20)
#                 Add_btn.grid(row=7,column=0, sticky="nsew")
                edit_btn = ttk.Button(self, text="Edit", command=edit,width=20)
                edit_btn.grid(row=7,column=1, sticky="nsew")
                del_btn = ttk.Button(self, text="Delete", command=delete)
                del_btn.grid(row=7,column=2, sticky="nsew")
                

        def delete():
           # Get selected item to Delete
            selected_item = tree1.selection()[0]
            
            print(tree1.item(selected_item)['values'][0])
            db=DatabaseConnectivity()
            db.DeleteAreacodeDetails(tree1.item(selected_item)['values'][0])
            tree1.delete(selected_item)
            tk.messagebox.showinfo("Record Deleted", "Record deleted Sucessfully")
   
            
             
            

        # Add Buttons to Edit and Delete the Treeview items

        edit_btn = ttk.Button(self, text="Edit", command=edit,width=20)
        edit_btn.grid(row=7,column=1, sticky="nsew")
        del_btn = ttk.Button(self, text="Delete", command=delete)
        del_btn.grid(row=7,column=2, sticky="nsew")
        
        def updatetatodb(AreaName,Areaserved,contcount,voilationcount,zipval,AreacodeIDval,precautionentry,hardnessentry):
            db=DatabaseConnectivity()
            print('areacodeid--')
            print('areacodeid--'+AreacodeIDval)
            addareacode= db.UpdateAreacodeDetails(AreaName,Areaserved,contcount,voilationcount,zipval,AreacodeIDval,precautionentry,hardnessentry)
            print(addareacode)
            tk.messagebox.showinfo("Record Update", "Record Updated Sucessfully")
            
        
        
            
        
            

        
class RegisterNewUser(tk.Frame):

    

    def __init__(self, parent, controller,arg):
        tk.Frame.__init__(self, parent)
        print('---')
        print(arg)
        self.controller = controller
        self.title='User Search'
        lblregisterheader = tk.Label(self, text="Register New User")
        lblregisterheader.grid(column=2, row=0, sticky=(N, S, E, W),padx=100)
        #lblregisterheader.pack(side="top", fill="x",padx=5,pady=10)
        #label2 = tk.Label(self, text=arg, font=controller.title_font)
        #label2.pack(side="top", fill="x", pady=10)
        #inputtxt1 = tk.Text(self, height = 10,
                #width = 25,
                #bg = "light yellow")
        #if(arg!=None):
            #inputtxt1.insert('end', arg[0])
        #inputtxt1.pack()
        firstname = StringVar()
        lastname = StringVar()
        email = StringVar()
        phoneno = StringVar()
        username = StringVar()
        password = StringVar()
        button = tk.Button(self,height = 1, text="Go to the Admin page",
                           command=lambda: controller.show_frame("AdminSearch",['']))
        #button.pack(padx=5,pady=3)
        button.grid(column=5, row=0, sticky=(N, S, E, W),padx=80)
        lblFirstName = tk.Label(self, text="First Name")
        
        lblFirstName.grid(column=1, row=2, sticky=(N, S, E, W))
        Firstname_entry = Entry(self,textvariable=firstname)
        
        Firstname_entry.grid(column=2, row=2, sticky=(N, S, E, W))
        lblLastName = tk.Label(self, text="Last Name")
       
        lblLastName.grid(column=1, row=3, sticky=(N, S, E, W))
        
        Lastname_entry = Entry(self,textvariable=lastname)
        
        Lastname_entry.grid(column=2, row=3, sticky=(N, S, E, W))
        
        lblemail = tk.Label(self, text="Email")
        
        lblemail.grid(column=1, row=4, sticky=(N, S, E, W))
        email_entry = Entry(self,textvariable=email)
        
        email_entry.grid(column=2, row=4, sticky=(N, S, E, W))
        
        lblphone = tk.Label(self, text="Phone")
        #lblphone.pack(side="top", fill="x")
        lblphone.grid(column=1, row=5, sticky=(N, S, E, W))
        phone_entry = Entry(self,textvariable=phoneno)
        #phone_entry.pack(padx=20,pady=10)
        phone_entry.grid(column=2, row=5, sticky=(N, S, E, W))
        
        lblusername = tk.Label(self, text="Username")
        #lblusername.pack(side="top", fill="x")
        lblusername.grid(column=1, row=6, sticky=(N, S, E, W))
        username_entry = Entry(self,textvariable=username)
        #username_entry.pack(padx=20,pady=10)
        username_entry.grid(column=2, row=6, sticky=(N, S, E, W))
        
        lblpassword = tk.Label(self, text="Password")
        #lblpassword.pack(side="top", fill="x")
        lblpassword.grid(column=1, row=7, sticky=(N, S, E, W))
        password_entry = Entry(self,textvariable=password)
        #password_entry.pack(padx=20,pady=10)
        password_entry.grid(column=2, row=7, sticky=(N, S, E, W))
        
        
        SaveUser_btn = ttk.Button(self, text="Submit", command=lambda:self.subdata(firstname.get(),lastname.get(),email.get(),phoneno.get(),username.get(),password.get()))
        #SaveUser_btn.pack()
        SaveUser_btn.grid(column=2, row=9, sticky=(N, S, E, W))
        
    def subdata(self,fisrtname,lastname,email,phoneno,username,password):
        db=DatabaseConnectivity()
        adduser= db.Adduser(fisrtname,lastname,email,phoneno,username,password)
        tk.messagebox.showinfo("User Details", "New user has been added successfully")
        
           
   
   
        
class DatabaseConnectivity():
    @staticmethod 
    def getconnection():
        return {"database": "waterquality", "user":"root","password":"Unlv@123"}
        
    
    def login_verification(self,username,password):
        print(username)
        print(password)
        try:
            
            
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            query = ("SELECT id,UserName FROM waterquality.user WHERE UserName = %s and password = %s ")
            num_items=cursor.execute(query ,(username, password))
            
            #cursor.execute(query)
            print('--rc')
            cursor.fetchall()
            if(cursor.rowcount==0):
                messagebox.showerror("showerror", "Not a valid login details")
                
            for (id,UserName) in cursor:
                print(id)
                print(UserName)
    
            cursor.close()
            cnx.close()
            print('a')
            
    @staticmethod        
    def fetchArecodedata(Zipcode):
        num_items=[]
        sdd=[]
        
        try:
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
                            
                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            query=("select Ac.AreacodeId,Ac.Zipcodes,AC.name,Ac.Area,vd.ViolationCount,vd.ContaminantCount,vd.precautions,vd.HardnessLevel from waterquality.areacodedetails as AC inner join waterquality.violationdata as  vd   on vd.AreaCodeID=AC.AreaCodeID  where AC.Zipcodes = %s ")
            #query = ("SELECT id,UserName FROM water.user WHERE UserName = %s and password = %s ")
            num_items=cursor.execute(query ,(Zipcode,))
            
            #cursor.execute(query)
            num_items=cursor.fetchall()          
            names = [ x[0] for x in cursor.description]
            sdd=pd.DataFrame( num_items, columns=names)
            sdd.set_index('Zipcodes')

            print(sdd)
            #print(sql_data)

            
    
            cursor.close()
            cnx.close()
            #print(num_items)
            return sdd
        
    @staticmethod
    def historcTrendsUpload(filename):
        a_list = []
        df = pd.read_csv (filename)
        print (df)
        
        mydb = mysql.connector.connect(user='root',password='Unlv@123',host='127.0.0.1',database='waterquality')
        query = 'select AreaCodeID, Zipcodes,Area from waterquality.AreacodeDetails '
        df1 = pd.read_sql(query, con = mydb)
        
        print(df1)
        df1['Zipcodes']=df1['Zipcodes'].astype(int)
        inner_join_df=df1.merge(df, left_on=['Area','Zipcodes'], right_on=['AreaName','Zipcode'])
        print(inner_join_df)
        inner_join_df.fillna(0)
        for index, row in inner_join_df.iterrows():
            print(row['AreaCodeID'])
            record1 = (row['AreaCodeID'],row['Year'],row['HardnessLevel'],row['ContaminantCount'],row['ViolationCount'])
            a_list.append(record1)
        try:       
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            sql = "delete  from waterquality.ViolationHistoricalData "
            cursor.execute(sql)
            mySql_insert_query1 = """INSERT INTO waterquality.ViolationHistoricalData (AreaCodeID,Year,HardnessLevel,ViolationCount,ContaminationCount) 
                                            VALUES (%s, %s, %s, %s,%s) """
            cursor.executemany(mySql_insert_query1, a_list)
            cnx.commit()
            cursor.execute("SELECT * from waterquality.ViolationHistoricalData")
            print(cursor.fetchall())
            cursor.close()
            cnx.close()

    @staticmethod        
    def fetchtrends(Zipcode):
        num_items=[]
        sdd=[]
        
        try:
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
                                
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            query=("select Ac.AreacodeId,Ac.Zipcodes,Ac.Area,vd.ViolationCount,vd.ContaminationCount ,vd.Year,vd.HardnessLevel from waterquality.AreacodeDetails as AC inner join waterquality.ViolationHistoricalData as  vd  on vd.AreaCodeId=AC.AreacodeId where zipcodes = %s ")
            #query = ("SELECT id,UserName FROM water.user WHERE UserName = %s and password = %s ")
            num_items=cursor.execute(query ,(Zipcode,))
            
            #cursor.execute(query)
            num_items=cursor.fetchall()          
            names = [ x[0] for x in cursor.description]
            sdd=pd.DataFrame( num_items, columns=names)
            sdd.set_index('Zipcodes')
            sdd.head().style.hide_index()

            
            #print(sql_data)
            
            cursor.close()
            cnx.close()
            #print(num_items)
            return sdd
    @staticmethod        
    def updateAreacodeDetails(Zipcode):
        num_items=[]
        sdd=[]
        
        try:
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            query=("select Ac.AreacodeId,Ac.Zipcodes,Ac.Area,vd.ViolationCount,vd.ContaminationCount ,vd.Year,vd.HardnessLevel from waterquality.AreacodeDetails as AC inner join waterqualiy.ViolationHistoricalData as  vd  on vd.AreaCodeId=AC.AreacodeId where zipcodes = %s ")
            #query = ("SELECT id,UserName FROM water.user WHERE UserName = %s and password = %s ")
            cursor.close()
            cnx.close()
    @staticmethod        
    def Adduser(firstname,lastname,email,phone,username,password):
        print(firstname)
        print(lastname)
        print(phone)
        print(username)
        print(password)
        
        try:
            #cnx = mysql.connector.connect(user='root',password='Unlv@123',host='127.0.0.1',database='waterquality')
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
            
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            lastrowid=0
            cursor = cnx.cursor()
            mySql_insert_query = """INSERT INTO waterquality.useradmindetails (UserName,Password,FirstName,LastName,Email,PhoneNumber) 
                                VALUES (%s, %s,%s,%s,%s,%s) """

            record = (username,password,firstname,lastname,email,phone)
            cursor.execute(mySql_insert_query, record)
            cnx.commit()
            cursor.close()
            cnx.close()      
            
    @staticmethod        
    def AddAreacodeDetails(AreaName,Areaserved,contcount,voilationcount,zipval,precautionentry,hardnessentryval):
        print(AreaName)
        print(Areaserved)
        print(contcount)
        print(voilationcount)
        print(zipval)
        print(precautionentry)
        
        try:
            #cnx = mysql.connector.connect(user='root',password='Unlv@123',host='127.0.0.1',database='waterquality')
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
            
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            lastrowid=0
            cursor = cnx.cursor()

            mySql_insert_query = """INSERT INTO waterquality.areacodedetails (Name,Area,Zipcodes) 
                                VALUES (%s, %s,%s) """

            record = (AreaName,Areaserved,zipval)
            cursor.execute(mySql_insert_query, record)
            cursor.lastrowid
            record1 = (cursor.lastrowid,voilationcount,contcount,precautionentry,hardnessentryval)
            mySql_insert_query1 = """INSERT INTO waterquality.ViolationData (AreaCodeId,ViolationCount,contaminantCount,Precautions,Hardnesslevel) 
                                VALUES (%s, %s, %s, %s,%s) """
            cursor.execute(mySql_insert_query1, record1)
                
            cnx.commit()
            
            
            cursor.close()
            cnx.close()
    @staticmethod        
    def UpdateAreacodeDetails(AreaName,Areaserved,contcount,voilationcount,zipval,AreaCodeId,precautions,hardnesscount):
        print(AreaName)
        print(Areaserved)
        print(contcount)
        print(voilationcount)
        print(zipval)
        print(AreaCodeId)
        print(hardnesscount)
        
        try:
            #cnx = mysql.connector.connect(user='root',password='Unlv@123',host='127.0.0.1',database='waterquality')
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
    
            cursor = cnx.cursor()
            record = (voilationcount,contcount,precautions,hardnesscount,AreaCodeId)
            mySql_update_query = """UPDATE waterquality.violationdata SET ViolationCount= %s,ContaminantCount=%s ,precautions =%s ,HardnessLevel =%s WHERE AreacodeId= %s """
            cursor.execute(mySql_update_query,record)
            cnx.commit()
            cursor.close()
            cnx.close()
    
    @staticmethod        
    def DeleteAreacodeDetails(Areacode):
        print(Areacode)
        
        
        try:
            #cnx = mysql.connector.connect(user='root',password='Unlv@123',host='127.0.0.1',database='waterquality')
            dbs=DatabaseConnectivity()
            dbconfig = dbs.getconnection()
            cnx = mysql.connector.connect(**dbconfig)
                                 
                                
        except mysql.connector.Error as err:
            if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
                print("Something is wrong with your user name or password")
            elif err.errno == errorcode.ER_BAD_DB_ERROR:
                print("Database does not exist")
            else:
                print(err)
        else:
            cursor = cnx.cursor()
            query = (" Delete from waterquality.violationdata where AreacodeId = %s")
            query2 = (" Delete from waterquality.violationhistoricaldata where AreaCodeId = %s")
            query1=(" Delete from waterquality.AreacodeDetails where AreacodeId = %s ")
            cursor.execute(query,(Areacode,))
            cursor.execute(query2,(Areacode,))
            cursor.execute(query1, (Areacode,))
            cnx.commit()
            #query = ("SELECT id,UserName FROM water.user WHERE UserName = %s and password = %s ")
            cursor.close()

# define a class for creating tables and validating existing tables
class tableMetadata:
    global conn
    @staticmethod
    def getconnection():
        return {"database": "waterquality", "user":"root","password":"Unlv@123"}
    # create a connect object
  
    
   
    @staticmethod
    def db_creation():
        # Create a cursor
        conn = mysql.connector.connect( user="root",password="Unlv@123")
        cur = conn.cursor()
        #create a database name waterquality monitoring
        DB_CREATE_STR = 'CREATE DATABASE IF NOT EXISTS waterquality'
        #use database for DML operations
        USE_DB_STR='USE waterquality'
        cur.execute(DB_CREATE_STR)
        cur.execute(USE_DB_STR)
        conn.commit()
        print('Database created')
    #define a static function to validate if the tables are existing or not
    @staticmethod
    def table_validation(table_name,  table_schema):
        #establishing cursor connection
        conn = mysql.connector.connect(**tableMetadata.getconnection())
        cur = conn.cursor()
        cur.execute("""SELECT count(*) FROM information_schema.tables 
        WHERE table_name = %s and  table_schema = %s """,(table_name,table_schema))
        #check if-else condition, if the table exits return true else false
        if cur.fetchone()[0] == 1:
                return True
        else:
            return False
    #define,a function to create tables in the database
    @staticmethod           
    def table_Creation():
        #calling the instance of the class defined
        obj=tableMetadata()
        conn = mysql.connector.connect(**tableMetadata.getconnection())
        cur = conn.cursor()
        if(obj.table_validation('useradmindetails','waterquality')==False):
            #create table useradmin details
            TABLE_ADMIN_STR= "CREATE TABLE `useradmindetails` (\
                              `Id` INT NOT NULL AUTO_INCREMENT,\
                              `UserName` VARCHAR(45) NOT NULL,\
                              `Password` VARCHAR(20) NOT NULL,\
                              `FirstName` VARCHAR(45) NOT NULL,\
                              `LastName` VARCHAR(45) NOT NULL,\
                              `Email` VARCHAR(45) NOT NULL,\
                              `PhoneNumber` VARCHAR(12) NOT NULL,PRIMARY KEY (`Id`))"
            cur.execute(TABLE_ADMIN_STR)
            print("table created")
            
         #create areacode details table
        if(obj.table_validation('areacodedetails','waterquality')==False):
            TABLE_AREACODEDETAILS_STR ='CREATE TABLE `areacodedetails` (\
                                            `AreaCodeID` int NOT NULL AUTO_INCREMENT,\
                                            `Zipcodes` varchar(10) NOT NULL,\
                                            `Area` varchar(100) NOT NULL,\
                                            `Name` varchar(100) NOT NULL,\
                                            PRIMARY KEY (`AreaCodeID`))'
            cur.execute(TABLE_AREACODEDETAILS_STR)
        #create historical data table
        if(obj.table_validation('violationhistoricaldata','waterquality')==False):
            TABLE_HISTORICDATA_STR = "  CREATE TABLE`violationhistoricaldata` (\
                                        `Id` int NOT NULL AUTO_INCREMENT,\
                                        `AreaCodeId` int NOT NULL,\
                                        `Year` int NOT NULL,\
                                        `HardnessLevel` int NOT NULL,\
                                        `ViolationCount` int NOT NULL,\
                                        `ContaminationCount` int NOT NULL,\
                                        PRIMARY KEY (`Id`),\
                                        INDEX `AreaCodeId_idx` (`AreaCodeId` ASC) VISIBLE,\
                                        CONSTRAINT `AreaCodeId` FOREIGN KEY (`AreaCodeId`) REFERENCES `areacodedetails` (`AreaCodeID`))"
            cur.execute(TABLE_HISTORICDATA_STR)                   
            #create violation data table
        if(obj.table_validation('violationdata','waterquality')==False):
            TABLE_VIOLATION_STR = 'CREATE TABLE `violationdata` (\
                                          `ViolationId`  int NOT NULL AUTO_INCREMENT,\
                                          `AreaCodeId` INT NOT NULL,\
                                          `ViolationCount` INT NOT NULL,\
                                          `ContaminantCount` INT NOT NULL,\
                                          `HardnessLevel` INT NOT NULL,\
                                          `Precautions` VARCHAR(400) NOT NULL,\
                                          PRIMARY KEY (`ViolationId`),\
                                          INDEX `AreaCodeId_idx` (`AreaCodeId` ASC) VISIBLE,\
                                          CONSTRAINT `AreaCodeId1`FOREIGN KEY (`AreaCodeID`)\
                                          REFERENCES `areacodedetails` (`AreaCodeID`))'
            
            cur.execute(TABLE_VIOLATION_STR)
            #AddAreacodeDetails(AreaName,Areaserved,contcount,voilationcount,zipval,precautionentry,hardnessentryval)
            ViolationinitialData = [
                                    {
                                    "AreaName": "Blue Diamond",
                                    "Contaminantcount": 45,
                                    "ViolationCount": 2,
                                    "HardnessLevel": 4,
                                    "Zipcode": 89004,
                                    "precaution": "single tank water softer",
                                    "Name": "Blue Diamond Watersupply"
                                    },
                                    {
                                    "AreaName": "Boulder City",
                                    "Contaminantcount": 42,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 6,
                                    "Zipcode": 89005,
                                    "precaution": "single tank water softer",
                                    "Name": "Boulder City Watersupply"
                                    },
                                    {
                                    "AreaName": "Boulder City Lake Mead Recreational Area",
                                    "Contaminantcount": 52,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89006,
                                    "precaution": "twin tank water softner",
                                    "Name": "Boulder City Lake Mead Recreational AreaWatersupply"
                                    },
                                    {
                                    "AreaName": "City of Henderson",
                                    "Contaminantcount": 20,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 3,
                                    "Zipcode": 89010,
                                    "precaution": "no precautions",
                                    "Name": "City of HendersonWatersupply"
                                    },
                                    {
                                    "AreaName": "City of Henderson Henderson",
                                    "Contaminantcount": 34,
                                    "ViolationCount": 1,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89011,
                                    "precaution": "no precautions",
                                    "Name": "City of Henderson Watersupply"
                                    },
                                    {
                                    "AreaName": "McDonald Ranch Neighborhood",
                                    "Contaminantcount": 76,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89012,
                                    "precaution": "single tank water softer",
                                    "Name": "McDonald Ranch Neighborhood Watersupply"
                                    },
                                    {
                                    "AreaName": "Henderson",
                                    "Contaminantcount": 54,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89014,
                                    "precaution": "no precautions",
                                    "Name": "Henderson Watersupply"
                                    },
                                    {
                                    "AreaName": "Henderson",
                                    "Contaminantcount": 8,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89015,
                                    "precaution": "twin tank water softner",
                                    "Name": "Henderson Watersupply"
                                    },
                                    {
                                    "AreaName": "North Las Vegas",
                                    "Contaminantcount": 65,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 13,
                                    "Zipcode": 89030,
                                    "precaution": "twin tank water softner",
                                    "Name": "North Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "North Las Vegas",
                                    "Contaminantcount": 43,
                                    "ViolationCount": 5,
                                    "HardnessLevel": 12,
                                    "Zipcode": 89031,
                                    "precaution": "twin tank water softner",
                                    "Name": "North Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "North Las Vegas",
                                    "Contaminantcount": 34,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89032,
                                    "precaution": "single tank water softer",
                                    "Name": "North Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Henderson",
                                    "Contaminantcount": 11,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 4,
                                    "Zipcode": 89044,
                                    "precaution": "single tank water softer",
                                    "Name": "Henderson Watersupply"
                                    },
                                    {
                                    "AreaName": "Henderson",
                                    "Contaminantcount": 12,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 6,
                                    "Zipcode": 89052,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply Henderson"
                                    },
                                    {
                                    "AreaName": "Aliante Master Planned Community",
                                    "Contaminantcount": 96,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89084,
                                    "precaution": "twin tank water softner",
                                    "Name": "Aliante Master Planned Community Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas Neighborhoods",
                                    "Contaminantcount": 90,
                                    "ViolationCount": 6,
                                    "HardnessLevel": 3,
                                    "Zipcode": 89102,
                                    "precaution": "no precautions",
                                    "Name": "Southwest Las Vegas Neighborhoods Watersupply"
                                    },
                                    {
                                    "AreaName": "East Las Vegas Neighborhoods",
                                    "Contaminantcount": 90,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89101,
                                    "precaution": "no precautions",
                                    "Name": "East Las Vegas Neighborhoods Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 87,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89103,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply Southwest Las Vegas"
                                    },
                                    {
                                    "AreaName": "Southeast Las Vegas",
                                    "Contaminantcount": 87,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89104,
                                    "precaution": "no precautions",
                                    "Name": "Southeast Watersupply Las Vegas"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas",
                                    "Contaminantcount": 85,
                                    "ViolationCount": 3,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89106,
                                    "precaution": "twin tank water softner",
                                    "Name": "Watersupply Northwest Las Vegas"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas",
                                    "Contaminantcount": 85,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 13,
                                    "Zipcode": 89107,
                                    "precaution": "twin tank water softner",
                                    "Name": "Northwest Watersupply  Las Vegas"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas",
                                    "Contaminantcount": 83,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 12,
                                    "Zipcode": 89108,
                                    "precaution": "twin tank water softner",
                                    "Name": "Northwest Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "South Las Vegas",
                                    "Contaminantcount": 80,
                                    "ViolationCount": 4,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89109,
                                    "precaution": "single tank water softer",
                                    "Name": "South Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "East Las Vegas",
                                    "Contaminantcount": 75,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 4,
                                    "Zipcode": 89110,
                                    "precaution": "single tank water softer",
                                    "Name": "East Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 42,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 6,
                                    "Zipcode": 89113,
                                    "precaution": "single tank water softer",
                                    "Name": "Southwest Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "East Las Vegas",
                                    "Contaminantcount": 52,
                                    "ViolationCount": 1,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89115,
                                    "precaution": "twin tank water softner",
                                    "Name": "East Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "The Lakes",
                                    "Contaminantcount": 20,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 3,
                                    "Zipcode": 89117,
                                    "precaution": "no precautions",
                                    "Name": "The Lakes Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 34,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89118,
                                    "precaution": "no precautions",
                                    "Name": "Southwest Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "South Las Vegas",
                                    "Contaminantcount": 76,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89119,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply South Las Vegas"
                                    },
                                    {
                                    "AreaName": "South Las Vegas",
                                    "Contaminantcount": 54,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89120,
                                    "precaution": "no precautions",
                                    "Name": "South Watersupply Las Vegas"
                                    },
                                    {
                                    "AreaName": "South Las Vegas",
                                    "Contaminantcount": 8,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89121,
                                    "precaution": "twin tank water softner",
                                    "Name": "South Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "East Las Vegas",
                                    "Contaminantcount": 65,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 13,
                                    "Zipcode": 89122,
                                    "precaution": "twin tank water softner",
                                    "Name": "East Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "South Las Vegas",
                                    "Contaminantcount": 43,
                                    "ViolationCount": 8,
                                    "HardnessLevel": 12,
                                    "Zipcode": 89123,
                                    "precaution": "twin tank water softner",
                                    "Name": "South Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas Desert Shores",
                                    "Contaminantcount": 34,
                                    "ViolationCount": 7,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89128,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply Desert Shores"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas",
                                    "Contaminantcount": 11,
                                    "ViolationCount": 6,
                                    "HardnessLevel": 4,
                                    "Zipcode": 89129,
                                    "precaution": "single tank water softer",
                                    "Name": "Northwest Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Summerlin Las Vegas The Trails Sun City Summerlin",
                                    "Contaminantcount": 12,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 6,
                                    "Zipcode": 89134,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply The Trails Sun City Summerlin"
                                    },
                                    {
                                    "AreaName": "Summerlin Las Vegas",
                                    "Contaminantcount": 96,
                                    "ViolationCount": 3,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89135,
                                    "precaution": "twin tank water softner",
                                    "Name": "Summerlin Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Summerlin Las Vegas",
                                    "Contaminantcount": 90,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 3,
                                    "Zipcode": 89138,
                                    "precaution": "no precautions",
                                    "Name": "Summerlin Watersupply Las Vegas"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 42,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89139,
                                    "precaution": "no precautions",
                                    "Name": "Southwest Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Southern Highlands, Las Vegas",
                                    "Contaminantcount": 52,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89141,
                                    "precaution": "single tank water softer",
                                    "Name": "Southern Highlands Watersupply"
                                    },
                                    {
                                    "AreaName": "East Las Vegas, Las Vegas",
                                    "Contaminantcount": 20,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89142,
                                    "precaution": "no precautions",
                                    "Name": "East Las Vegas Watersupply"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas",
                                    "Contaminantcount": 34,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89145,
                                    "precaution": "twin tank water softner",
                                    "Name": "Northwest Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 76,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 13,
                                    "Zipcode": 89146,
                                    "precaution": "twin tank water softner",
                                    "Name": "Southwest Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 54,
                                    "ViolationCount": 5,
                                    "HardnessLevel": 12,
                                    "Zipcode": 89147,
                                    "precaution": "twin tank water softner",
                                    "Name": "Southwest Watersupply"
                                    },
                                    {
                                    "AreaName": "Rhodes Ranch, SW Las Vegas",
                                    "Contaminantcount": 8,
                                    "ViolationCount": 2,
                                    "HardnessLevel": 8,
                                    "Zipcode": 89148,
                                    "precaution": "single tank water softer",
                                    "Name": "Rhodes Ranch Watersupply"
                                    },
                                    {
                                    "AreaName": "Northwest Las Vegas, Centennial Hills",
                                    "Contaminantcount": 65,
                                    "ViolationCount": 7,
                                    "HardnessLevel": 4,
                                    "Zipcode": 89149,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply Centennial Hills"
                                    },
                                    {
                                    "AreaName": "South Las Vegas",
                                    "Contaminantcount": 43,
                                    "ViolationCount": 9,
                                    "HardnessLevel": 6,
                                    "Zipcode": 89169,
                                    "precaution": "single tank water softer",
                                    "Name": "Watersupply South Las Vegas"
                                    },
                                    {
                                    "AreaName": "Mountains Edge Area, Southwest Las Vegas",
                                    "Contaminantcount": 34,
                                    "ViolationCount": 1,
                                    "HardnessLevel": 11,
                                    "Zipcode": 89178,
                                    "precaution": "twin tank water softner",
                                    "Name": "Mountains Edge Area Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 11,
                                    "ViolationCount": 0,
                                    "HardnessLevel": 3,
                                    "Zipcode": 89179,
                                    "precaution": "no precautions",
                                    "Name": "Southwest Watersupply"
                                    },
                                    {
                                    "AreaName": "Southwest Las Vegas",
                                    "Contaminantcount": 12,
                                    "ViolationCount": 3,
                                    "HardnessLevel": 2,
                                    "Zipcode": 89183,
                                    "precaution": "no precautions",
                                    "Name": "Watersupply Southwest"
                                    }]
                                       
                                        
            db=DatabaseConnectivity()
            db.Adduser('admin','admin','admin@wm.com','1112223333','admin','admin')   
            for i in range(len(ViolationinitialData)):
                #print(ViolationinitialData[i]['ContaminantCount'])
                db.AddAreacodeDetails(ViolationinitialData[i]['AreaName'],ViolationinitialData[i]['Name'],ViolationinitialData[i]['Contaminantcount'],ViolationinitialData[i]['ViolationCount'],ViolationinitialData[i]['Zipcode'],ViolationinitialData[i]['precaution'],ViolationinitialData[i]['HardnessLevel'])
                            
            
                        
                            
                            
            
            
            
        cur.close()          

if __name__ == "__main__":
    app = WaterQualityManagementApp()
    app.title(title)
    app.minsize(300,300)
    app.geometry("800x600")
    app.mainloop()








StartPage
UserSearchPage
AdminSearch
AdminLogin
RegisterNewUser
---
None
Database created
table created
admin
admin
1112223333
admin
admin
Blue Diamond
Blue Diamond Watersupply
45
2
89004
single tank water softer
Boulder City
Boulder City Watersupply
42
0
89005
single tank water softer
Boulder City Lake Mead Recreational Area
Boulder City Lake Mead Recreational AreaWatersupply
52
0
89006
twin tank water softner
City of Henderson
City of HendersonWatersupply
20
0
89010
no precautions
City of Henderson Henderson
City of Henderson Watersupply
34
1
89011
no precautions
McDonald Ranch Neighborhood
McDonald Ranch Neighborhood Watersupply
76
0
89012
single tank water softer
Henderson
Henderson Watersupply
54
0
89014
no precautions
Henderson
Henderson Watersupply
8
0
89015
twin tank water softner
North Las Vegas
North Las Vegas Watersupply
65
0
89030
twin tank water softner
North Las Vegas
North Las Vegas Watersupply
43
5
89031
twin tank water softner
North Las Vegas
North Las Vegas Watersuppl

### 